In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.mixture import GaussianMixture
from sklearn.cluster import Birch
from yellowbrick.cluster import KElbowVisualizer

import warnings 
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
data = pd.read_csv('../input/dataset-of-songs-in-spotify/genres_v2.csv')
#data.head(10)
display(data.columns)
#刪除多餘column
data.drop(columns=['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms','genre', 'song_name', 'Unnamed: 0', 'title'],axis=1, inplace=True)
print("--------------------------------------------------------------------------------------------------------------------------------------")
data.info()
data
print("--------------------------------------------------------------------------------------------------------------------------------------")
#find missing value
data.isnull().sum()

In [ ]:
#Feature Normalization
scaling=StandardScaler()
scaled=scaling.fit_transform(data)
scaled_df=pd.DataFrame(scaled,columns=data.columns)

In [ ]:
#deal with correct data
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data2 = pd.read_csv('../input/dataset-of-songs-in-spotify/genres_v2.csv')
display(data2.columns)
data2.drop(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms','time_signature', 'song_name', 'Unnamed: 0', 'title'],axis=1, inplace=True)
data2['genre'] = labelencoder.fit_transform(data2['genre'])
print(data2)
scaling=StandardScaler()
scaled2=scaling.fit_transform(data2)
scaled_df2=pd.DataFrame(scaled2,columns=data2.columns)
scaled_df2

In [ ]:
#use select feature 
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder

sel_data = pd.read_csv('../input/dataset-of-songs-in-spotify/genres_v2.csv')
sel_data.drop(columns=['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'song_name', 'Unnamed: 0', 'title'],axis=1, inplace=True)

sel_data['genre'] = labelencoder.fit_transform(sel_data['genre'])

plt.figure(figsize=(10,8))
feature_corr = sel_data.corr()
sns.heatmap(feature_corr,annot=True)

sel_data.drop(columns=["time_signature","genre"],axis=1, inplace=True)
sel_data.info()

scaling=StandardScaler()
scaled=scaling.fit_transform(sel_data)
scaled_sel_df=pd.DataFrame(scaled,columns=sel_data.columns)
print(scaled_sel_df)

In [ ]:
#K-means with select feature 
#群數
kmeans = KMeans(n_clusters=3,random_state=0)
kmeans.fit(scaled_sel_df)
#print(len(kmeans.labels_))
scaled_sel_df['guess_genre']=kmeans.labels_
scaled_sel_df.to_excel('sample_data.xlsx', sheet_name='sheet1', index=False)
#print(scaled_sel_df)
#正確的分群結果
#kmeans_label = KMeans(n_clusters=15,random_state=0)
#kmeans_label.fit(scaled_df2)
#每群的數量
#num = pd.Series(kmeans.labels_).value_counts()
#print(num)
#print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#主要的音樂類別
#print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#Rand Index, Normalized Mutual Information, Adjusted Mutual Information, V-measure, Fowlkes-Mallows Scores, Silhouette Coefficient, Confusion Matrix.
#print("rand Index:",metrics.rand_score(kmeans_label.labels_, kmeans.labels_))
#print("Normalized Mutual Information:",metrics.normalized_mutual_info_score(kmeans_label.labels_,kmeans.labels_))
#print("Adjusted Mutual Information:",metrics.adjusted_mutual_info_score(kmeans_label.labels_,kmeans.labels_))
#print("V-measure:",metrics.v_measure_score(kmeans_label.labels_,kmeans.labels_))
#print("Fowlkes-Mallows Scores:",metrics.fowlkes_mallows_score(kmeans_label.labels_,kmeans.labels_))
#print("Silhouette Coefficient:",metrics.silhouette_score(scaled_df, kmeans.labels_))
#print("Confusion Matrix:")
#cm = confusion_matrix(np.arange(25), np.arange(25))
#disp = ConfusionMatrixDisplay.from_predictions(kmeans_label.labels_,kmeans.labels_)
#fig, ax = plt.subplots(figsize=(10,10))
#disp.plot(ax=ax)

In [ ]:
#K-means
start = time.process_time()

K_means_arr=[]
K=range(1,15)

for i in K:
    kmean=KMeans(n_clusters=i)
    kmean.fit(data)
    K_means_arr.append(kmean.inertia_)
  
plt.plot(K,K_means_arr,marker='o')
plt.title('Elbow Method',fontsize=15)
plt.xlabel('Number of clusters',fontsize=15)
plt.ylabel('Sum of Squared distance',fontsize=15)
plt.show()
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#群數
kmeans = KMeans(n_clusters=3,random_state=0)
kmeans.fit(scaled_df)
#output data
scaled_df['guess_genre']=kmeans.labels_
scaled_df.to_excel('k-means.xlsx', sheet_name='sheet1', index=False)
#正確的分群結果
kmeans_label = KMeans(n_clusters=15,random_state=0)
kmeans_label.fit(scaled_df2)
#每群的數量
num = pd.Series(kmeans.labels_).value_counts()
print(num)
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#主要的音樂類別
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#Rand Index, Normalized Mutual Information, Adjusted Mutual Information, V-measure, Fowlkes-Mallows Scores, Silhouette Coefficient, Confusion Matrix.
print("rand Index:",metrics.rand_score(kmeans_label.labels_, kmeans.labels_))
print("Normalized Mutual Information:",metrics.normalized_mutual_info_score(kmeans_label.labels_,kmeans.labels_))
print("Adjusted Mutual Information:",metrics.adjusted_mutual_info_score(kmeans_label.labels_,kmeans.labels_))
print("V-measure:",metrics.v_measure_score(kmeans_label.labels_,kmeans.labels_))
print("Fowlkes-Mallows Scores:",metrics.fowlkes_mallows_score(kmeans_label.labels_,kmeans.labels_))
print("Silhouette Coefficient:",metrics.silhouette_score(scaled_df, kmeans.labels_))
print("Confusion Matrix:")
cm = confusion_matrix(np.arange(25), np.arange(25))
disp = ConfusionMatrixDisplay.from_predictions(kmeans_label.labels_,kmeans.labels_)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)

end = time.process_time()
print("執行時間：%f 秒" % (end - start))

In [ ]:
#Hierarchical Clustering
start = time.process_time()

model = AgglomerativeClustering()

visualizer = KElbowVisualizer(model, k=(1,15), timings=False)
visualizer.fit(data)
visualizer.show()
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#群數
hier = AgglomerativeClustering(n_clusters = 3)
hier.fit(scaled_df)
#output data
scaled_df['guess_genre']=hier.labels_
scaled_df.to_excel('hier.xlsx', sheet_name='sheet1', index=False)
#正確的分群結果
hier_label = AgglomerativeClustering(n_clusters=15)
hier_label.fit(scaled_df2)
#每群的數量
num = pd.Series(hier.labels_).value_counts()
print(num)
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#主要的音樂類別
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#Rand Index, Normalized Mutual Information, Adjusted Mutual Information, V-measure, Fowlkes-Mallows Scores, Silhouette Coefficient, Confusion Matrix.
print("rand Index:",metrics.rand_score(hier_label.labels_, hier.labels_))
print("Normalized Mutual Information:",metrics.normalized_mutual_info_score(hier_label.labels_,hier.labels_))
print("Adjusted Mutual Information:",metrics.adjusted_mutual_info_score(hier_label.labels_,hier.labels_))
print("V-measure:",metrics.v_measure_score(hier_label.labels_,hier.labels_))
print("Fowlkes-Mallows Scores:",metrics.fowlkes_mallows_score(hier_label.labels_,hier.labels_))
print("Silhouette Coefficient:",metrics.silhouette_score(scaled_df, hier.labels_))
print("Confusion Matrix:")
cm = confusion_matrix(np.arange(25), np.arange(25))
disp = ConfusionMatrixDisplay.from_predictions(hier_label.labels_,hier.labels_)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)

end = time.process_time()
print("執行時間：%f 秒" % (end - start))

In [ ]:
# DBCASN Clustering
start = time.process_time()

plt.figure(figsize=(5,5))
nn = NearestNeighbors(n_neighbors=5).fit(data)
distances, idx = nn.kneighbors(data)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)
plt.show()
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#群數
dbs = DBSCAN(eps = 2, min_samples = 10)
dbs.fit(scaled_df)
#output data
scaled_df['guess_genre']=dbs.labels_
scaled_df.to_excel('dbs.xlsx', sheet_name='sheet1', index=False)
#正確的分群
dbs_label = DBSCAN(eps = 2, min_samples = 10)
dbs_label.fit(scaled_df2)
#每群的數量
num = pd.Series(dbs.labels_).value_counts()
print(num)
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#主要的音樂類別
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#Rand Index, Normalized Mutual Information, Adjusted Mutual Information, V-measure, Fowlkes-Mallows Scores, Silhouette Coefficient, Confusion Matrix.
print("rand Index:",metrics.rand_score(dbs_label.labels_, dbs.labels_))
print("Normalized Mutual Information:",metrics.normalized_mutual_info_score(dbs_label.labels_,dbs.labels_))
print("Adjusted Mutual Information:",metrics.adjusted_mutual_info_score(dbs_label.labels_,dbs.labels_))
print("V-measure:",metrics.v_measure_score(dbs_label.labels_,dbs.labels_))
print("Fowlkes-Mallows Scores:",metrics.fowlkes_mallows_score(dbs_label.labels_,dbs.labels_))
print("Silhouette Coefficient:",metrics.silhouette_score(scaled_df, dbs.labels_))
print("Confusion Matrix:")
cm = confusion_matrix(np.arange(25), np.arange(25))
disp = ConfusionMatrixDisplay.from_predictions(dbs_label.labels_,dbs.labels_)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)

end = time.process_time()
print("執行時間：%f 秒" % (end - start))

In [ ]:
#GMM
start = time.process_time()

#log_like_lst = []
#all_cluster = 15

#for k in range(2, all_cluster):
    #gmm = GaussianMixture(n_components = k, random_state = 100).fit(data)
    #log_like = gmm.bic(data)
    #log_like_lst.append(log_like)

#plt.plot(range(2, all_cluster), log_like_lst, alpha=0.5)
#plt.ylabel('BIC')
#plt.xlabel('K')
#plt.show()
#print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#df_cluster = data.copy()
#分群
gmm = GaussianMixture(n_components = 13).fit(scaled_df)
#output data
scaled_df['guess_genre']=gmm.predict(scaled_df)
scaled_df.to_excel('gmm.xlsx', sheet_name='sheet1', index=False)
#正確的分群結果
gmm_label = GaussianMixture(n_components=15).fit(scaled_df2)
#每群的數量
#num = pd.Series(gmm.predict(scaled_df)).value_counts()
#print(num)
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#Rand Index, Normalized Mutual Information, Adjusted Mutual Information, V-measure, Fowlkes-Mallows Scores, Silhouette Coefficient, Confusion Matrix.
print("rand Index:",metrics.rand_score(gmm_label.predict(scaled_df2), gmm.predict(scaled_df)))
print("Normalized Mutual Information:",metrics.normalized_mutual_info_score(gmm_label.predict(scaled_df2),gmm.predict(scaled_df)))
print("Adjusted Mutual Information:",metrics.adjusted_mutual_info_score(gmm_label.predict(scaled_df2),gmm.predict(scaled_df)))
print("V-measure:",metrics.v_measure_score(gmm_label.predict(scaled_df2),gmm.predict(scaled_df)))
print("Fowlkes-Mallows Scores:",metrics.fowlkes_mallows_score(gmm_label.predict(scaled_df2),gmm.predict(scaled_df)))
print("Silhouette Coefficient:",metrics.silhouette_score(scaled_df, gmm.predict(scaled_df)))
print("Confusion Matrix:")
cm = confusion_matrix(np.arange(25), np.arange(25))
disp = ConfusionMatrixDisplay.from_predictions(gmm_label.predict(scaled_df2),gmm.predict(scaled_df))
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)

end = time.process_time()
print("執行時間：%f 秒" % (end - start))

In [ ]:
#BIRCH
start = time.process_time()
model = Birch()

visualizer = KElbowVisualizer(model, k=(1,15), timings=False)
visualizer.fit(data)
visualizer.show()
#群數
birch = Birch(n_clusters=3)
birch.fit(scaled_df)
#正確的分群結果
birch_label = Birch(n_clusters=15)
birch_label.fit(scaled_df2)
#output data
scaled_df['guess_genre']=birch.labels_
scaled_df.to_excel('birch.xlsx', sheet_name='sheet1', index=False)
#每群的數量
num = pd.Series(birch.labels_).value_counts()
print(num)
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#主要的音樂類別
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#Rand Index, Normalized Mutual Information, Adjusted Mutual Information, V-measure, Fowlkes-Mallows Scores, Silhouette Coefficient, Confusion Matrix.
print("rand Index:",metrics.rand_score(birch_label.labels_, birch.labels_))
print("Normalized Mutual Information:",metrics.normalized_mutual_info_score(birch_label.labels_,birch.labels_))
print("Adjusted Mutual Information:",metrics.adjusted_mutual_info_score(birch_label.labels_,birch.labels_))
print("V-measure:",metrics.v_measure_score(birch_label.labels_,birch.labels_))
print("Fowlkes-Mallows Scores:",metrics.fowlkes_mallows_score(birch_label.labels_,birch.labels_))
print("Silhouette Coefficient:",metrics.silhouette_score(scaled_df, birch.labels_))
print("Confusion Matrix:")
cm = confusion_matrix(np.arange(25), np.arange(25))
disp = ConfusionMatrixDisplay.from_predictions(birch_label.labels_,birch.labels_)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)

end = time.process_time()
print("執行時間：%f 秒" % (end - start))